In [11]:
from ppo_single_asset import BitcoinTradingEnv
from ppo_single_asset import ActorCritic
import torch
import numpy as np
import matplotlib.pyplot as plt
from ppo_single_asset import evaluate

test_data_ranges = {
        'train': ('2020-01-01', '2022-12-31'),
        'val': ('2023-01-01', '2023-02-01'),
        'test': ('2023-03-01', '2025-05-01')
}
env_test = BitcoinTradingEnv(
    csv_path="data/BTCUSDT_1h_full_history.csv",
    window_size=168,  # 7 days * 24 hours
    episode_length=1870,
    date_ranges_by_mode=test_data_ranges
)

/home/shuttle0615/RL_ONE/RL_repo/src/environment.py:315: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df = self.df.fillna(method='bfill').fillna(0)


In [12]:
input_shape = env_test.observation_space["market_data"].shape
n_actions = env_test.action_space.n

actor_critic = ActorCritic(input_shape, n_actions)

actor_critic.load_state_dict(torch.load("best_model_2.184_normalbest.pt"))

<All keys matched successfully>

In [15]:
test_reward, test_rewards, test_actions = evaluate(env_test, actor_critic, n_episodes=1, mode='test')
print(f"\nTest set evaluation mean reward: {np.exp(test_reward):.3f}") 
print(f"Test set rewards: {[np.exp(r[0]) for r in test_rewards]}")
#print(f"Test set reward changes: {[r[1] for r in test_rewards]}")

price_total = [r[1] for r in test_rewards]

# Set figure style
plt.style.use('seaborn')
fig, ax = plt.subplots(figsize=(15, 7))

# Get just the first episode
episode_returns = np.array(price_total[0], dtype=np.float64)
actual_returns = np.exp(episode_returns) - 1
cumulative_pnl = np.cumprod(1 + actual_returns) - 1

# Plot single line
ax.plot(cumulative_pnl * 100,  # multiply by 100 to show percentage
        label='Trading Performance',
        linewidth=2,
        color='#2E86C1')  # Nice blue color

# Customize the plot
ax.set_title('Cumulative PnL Over Time', fontsize=14, pad=15)
ax.set_xlabel('Trading Steps', fontsize=12)
ax.set_ylabel('Cumulative Return (%)', fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7)
ax.legend(fontsize=10)

# Add some margin to the plot
plt.margins(x=0.02)

# Customize tick labels
ax.tick_params(axis='both', labelsize=10)

# Show the plot
#plt.tight_layout()
plt.show()

100%|██████████| 1/1 [00:05<00:00,  5.67s/it]
/tmp/ipykernel_356585/2934266622.py:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')



Test set evaluation mean reward: 1.055
Test set rewards: [np.float64(1.0548524366020167)]


RecursionError: maximum recursion depth exceeded while calling a Python object